In [2]:
import pandas as pd
import numpy as np
import operator
from itertools import islice
import glob, os

In [3]:
pd.set_option('display.max_rows', None)

# Import hotel list and IMPD run catalogue
hotels = pd.read_csv('hotel_list.csv')
runcat = pd.read_csv('impd_run_catalogue.csv')

path = r'C:\Users\ajaworsk\Desktop\hotel_data\calls\2022'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
all_fd_files = glob.glob(os.path.join(path, "fd", "*.csv"))  
all_pd_files = glob.glob(os.path.join(path, "pd", "*.xls"))  

# Loop through files in both paths and append to appropriate list
fdlist = []
pdlist = []

for filename in all_fd_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    fdlist.append(df)
    
for filename in all_pd_files:
    df = pd.read_excel(filename, index_col=None, skiprows=1)
    pdlist.append(df)

# Concatenate both lists into dataframes, one for IFD runs and one for IMPD runs
firedf = pd.concat(fdlist, axis=0, ignore_index=True)
policedf = pd.concat(pdlist, axis=0, ignore_index=True)

# Remove white space from police column names
policedf.columns = policedf.columns.str.replace(' ', '')

# Remove ending white space from phone number column
policedf['CallerPhone'] = policedf['CallerPhone'].str.strip()

# Create 'CallerFullName' column to police dataframe
policedf.insert(5, 'CallerFullName', policedf['CallerFirstName'].fillna('') + ' ' + policedf['CallerLastName'].fillna(''))

# Merge police dataframe with IMPD run catalogue 
policedf = pd.merge(policedf,
                   runcat,
                   on = 'IncidentDescription',
                   how = 'inner')

# Sort fire and police dataframes by incident date in ascending order
firedf['IncidentDate'] = pd.to_datetime(firedf['IncidentDate'], format='%m/%d/%Y %I:%M:%S %p')
firedf = firedf.sort_values(by='IncidentDate', ascending=True).reset_index(drop=True)

policedf['IncidentDate'] = pd.to_datetime(policedf['IncidentDate'], format='%m/%d/%Y %I:%M:%S %p')
policedf = policedf.sort_values(by='IncidentDate', ascending=True).reset_index(drop=True)

WARNING *** file size (434112) not 512 + multiple of sector size (512)
WARNING *** file size (401344) not 512 + multiple of sector size (512)
WARNING *** file size (425920) not 512 + multiple of sector size (512)
WARNING *** file size (499648) not 512 + multiple of sector size (512)
WARNING *** file size (442304) not 512 + multiple of sector size (512)
WARNING *** file size (548800) not 512 + multiple of sector size (512)
WARNING *** file size (475072) not 512 + multiple of sector size (512)


In [4]:
# Common words in 'CallerFullName' column
commonWords = ['STAFF', 'DESK', 'CLERK', 'MANAGER', 'MGR', 'SUPERVISOR',
               'SUPV', 'EMPLOYEE', 'EMPLYEE', 'MAINTENANCE', 'SECURITY', 
               'INN', 'MOTEL', 'HOTEL', 'SUITE', 'HILTON', 'RED ROOF', 
               'MOTOR 8', 'EXTENDED', 'ZEE', 'PATEL', 'TOMMY DECKER', 'BENZ', 'IFD', 'ASHLEY HOLDEN']

# Employee phone numbers
hotelNumbers = ['317 2474100', '317 7423755', '614 3154737', '317 7725023', '317 2000784', '402 5945500', '317 7880331 0', 
                '317 7880331', '317 4942590', '317 7880333', '317 7880336', '317 7881124', '317 7880332', '317 7880335 0',
                '317 7880500', '317 7880335', '317 7378314', '541 5086434', '317 7861692', '317 9461329', '317 7861689', 
                '317 7861609', '765 9142622', '317 4880656', '317 3561036', '317 5472222', '317 5492222', '317 4508435',
                '720 6456237', '317 7355715', '317 2449558', '317 7289310', '317 8334142', '317 7551283', '317 3562389', 
                '317 3562388', '317 3561084', '317 2679469', '317 6947419', '317 9562182', '317 9810583', '317 7840047', 
                '317 6723391', '317 6250304', '317 6723378', '317 8220840', '317 7466360', '317 2133890', '317 6408775',
                '317 6849705', '317 2471480', '317 6520363', '317 6587885', '317 9860101', '317 7021990', '317 9980315', 
                '317 8095595', '317 7423611', '317 6352000', '317 5186891', '317 2652000', '317 3595341', '317 8727700', 
                '317 8226400', '317 7889551', '317 7831024', '317 3347891', '317 4291042', '317 7880811', '317 8221213', 
                '414 3349797', '317 7811350', '317 7576126', '317 2704554', '317 9929764', '402 3039811', '317 3893596', 
                '317 7576145', '317 5456056', '317 5456052', '317 5456055', '317 5456054', '317 5456051', '317 5456053', 
                '317 4861100', '317 3402592', '317 3904143', '603 7154195', '317 6344391', '603 7389092', '317 4172437', 
                '317 4861135', '317 4861134', '317 7575925', '317 4860158', '317 4860159', '317 7575939', '317 6613470', 
                '317 7315050', '317 9815270', '317 9696686', '317 6723781', '317 8029844', '765 2788225', '317 4143116', 
                '317 8273576', '317 2488362', '317 2488363', '502 4727436', '317 4696933', '317 2936552', '317 2936551', 
                '317 2936550', '317 2443361', '317 3598201', '317 3578074', '317 3520481', '317 7014214', '317 6297822',
                '812 4937656', '317 7569799', '317 8021128', '317 6797552', '317 8027110', '317 3312526', '765 2741141', 
                '317 8418585', '559 9724321', '317 4402057', '317 8412790', '317 9371028', '317 7884774', '317 8002417', 
                '317 8089872', '317 5606809', '317 5274639', '317 7229023', '317 7713305', '317 7229558', '317 6773477',
                '317 3841400', '317 9819547', '253 2594139', '317 6642622', '317 6539772', '317 5085797', '317 6520671',
                '317 4605413', '317 7957564', '317 7405260', '317 9148470', '317 7444723', '317 4061468', '317 2006883',
                '463 2361366', '317 7240452', '317 4135377', '317 7342081', '317 9852605', '317 7348476', '317 8301109',
                '317 7413774', '317 7198643', '317 2820374', '317 7504258', '463 2361317', '463 8882279', '317 4184479',
                '317 9652182', '317 8282956', '317 3535240', '413 8838462', '463 3331530', '317 2469785', '317 4487534',
                '317 6712161', '317 3667472', '317 6658928', '317 5149770', '317 4869941', '317 6680570', '317 9602865',
                '317 7027019', '317 4130805', '317 6900728']

regstr1 = '|'.join(commonWords)
regstr2 = '|'.join(hotelNumbers)

# Create column 'IsHotelStaff' 
# Searches for common words and phone numbers 
# If common word is found in 'CallerFullName' or number found in 'CallerPhone', then 'IsHotelStaff' value is True else False
policedf['IsHotelStaff'] = policedf['CallerFullName'].str.upper().str.contains(regstr1) | policedf['CallerPhone'].str.upper().str.contains(regstr2)

# If caller name is empty, caller is not hotel staff
#policedf['IsHotelStaff'] = np.where((policedf['CallerFullName'] == ' '), False, policedf['IsHotelStaff'])

# Create 'UsedInRatio' column based on conditions of 'CallForService' and 'IsHotelStaff' values
policedf['UsedInRatio'] = np.where((policedf['CallForService'] == True) & (policedf['IsHotelStaff'] == False), True, False)

# If caller number empty, do not use in ratio
policedf['UsedInRatio'] = np.where(pd.isna(policedf['CallerPhone']), False, policedf['UsedInRatio'])

# Manually change UsedInRatio values from True to False for these rows
extraFalseRows = [5115]
policedf['UsedInRatio'].iloc[extraFalseRows] = False

# Manually change UsedInRatio values from False to True for these rows
extraTrueRows = []
policedf['UsedInRatio'].iloc[extraTrueRows] = True

C:\Users\ajaworsk\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
# Dataframes of total fire runs, total police runs, and filtered police runs used in ratio
fireRuns = firedf['Location'].value_counts().rename_axis('Location').reset_index(name='TotalFireCalls')
policeRuns = policedf['Location'].value_counts().rename_axis('Location').reset_index(name='TotalPoliceCalls')
filteredPoliceRuns = policedf['Location'][(policedf['UsedInRatio'] == True)].value_counts().rename_axis('Location').reset_index(name="FilteredPoliceCalls")

# Merge dataframes
# Create 'RunsPerRoomRatio' column by taking (filtered police runs + total fire runs) / total rooms
# Sort by descending ratio
hotels = pd.merge(pd.merge(hotels, policeRuns, on='Location'), filteredPoliceRuns, on='Location')
hotels = pd.merge(hotels, fireRuns, on='Location')
hotels['CallsPerRoomRatio'] = round((hotels['FilteredPoliceCalls'] + hotels['TotalFireCalls']) / hotels['TotalRooms'], 2)
hotels = hotels.sort_values(by='CallsPerRoomRatio', ascending=False).reset_index(drop=True)

In [6]:
# Check if police reports contain incident not listed in run catalogue
reportedIncidents = list(policedf['IncidentDescription'].unique())
catIncidents = list(runcat['IncidentDescription'].unique())

for incident in reportedIncidents:
    if incident not in catIncidents:
        print(incident)

In [13]:
hotels.to_csv('hotelRatios2022.csv', index=False)

In [7]:
#Kartik Patel
policedf.loc[(policedf['Location'] == '7410 E 21ST ST') | (policedf['Location'] == '6850 E 21ST ST') | (policedf['Location'] == '6231 W WASHINGTON ST')].to_csv('policeRuns.csv', index=False)
firedf.loc[(firedf['Location'] == '7410 E 21ST ST') | (firedf['Location'] == '6850 E 21ST ST') | (firedf['Location'] == '6231 W WASHINGTON ST')].to_csv('fireRuns.csv', index=False)

In [7]:
hotels

,Name,Location,TotalRooms,TotalPoliceCalls,FilteredPoliceCalls,TotalFireCalls,CallsPerRoomRatio
0,COUNTRY HEARTH INN & SUITES,8850 E 21ST ST,80,255,132,93,2.81
1,PARK TERRACE MOTEL,9025 PENDLETON PIKE,22,60,35,16,2.32
2,SHADELAND INN,3525 N SHADELAND AVE,113,232,157,72,2.03
3,HARDING HOSPITALITY - MOTEL 6,4585 S HARDING ST,70,195,106,32,1.97
4,MOTOR 8 INN,3731 N SHADELAND AVE,160,348,194,78,1.70
5,GATEWAY MOTEL,1740 LAFAYETTE RD,48,118,48,33,1.69
6,AMERICA'S BEST VALUE INN,3401 S KEYSTONE AVE,65,210,66,31,1.49
7,MOTEL 6,2851 N SHADELAND AVE,117,246,114,58,1.47
8,NaN,11077 E WASHINGTON ST,20,35,14,14,1.40
9,SKYLINE MOTEL,6617 E WASHINGTON ST,61,95,51,34,1.39
